In [9]:
from dataset.dataset import ML1MDataset
from dataloader.dataloader import BertDataloader

In [3]:
from options import args
dataset = ML1MDataset(args)

In [10]:
dataset

In [12]:
dataset_path = dataset._get_preprocessed_dataset_path()
if not dataset_path.parent.is_dir():
    dataset_path.parent.mkdir(parents=True)

In [63]:
df = dataset.load_ratings_df()

raw_code's return : ml-1m


/Users/kyeongchanlee/PycharmProjects/BertCustom/BertCustom/dataset/dataset.py:59: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='::', header=None)


In [64]:
df = dataset.make_implicit(df)

Turning into implicit ratings


In [65]:
df = dataset.filter_triplets(df)

Filtering triplets


In [71]:
df

,uid,sid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [72]:
df, u_i_map = dataset.densify_index(df, target_cols=['sid', 'uid'])

Densifying index


In [74]:
tra_item_seq, val_item_seq, tes_item_seq = dataset.split_tra_val_tes(df, len(u_i_map['uid']))

Splitting


100%|██████████| 6040/6040 [00:02<00:00, 2423.07it/s]


In [75]:
dataset.attr_names

['genre']

In [79]:
def load_attrs_df(self):
    '''
    Return : 
        attr_df = itemid |   genre  |  ...
                    1    | 'comedy' |
    '''
    attr_df = dataset.load_movies_df()
    # attr_df = attr_df[attr_df['sid'].isin(rating_df['sid'])]
    return attr_df[['sid'] + dataset.attr_names]

In [80]:
attrs_df = load_attrs_df(df)

raw_code's return : ml-1m


/Users/kyeongchanlee/PycharmProjects/BertCustom/BertCustom/dataset/dataset.py:66: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, sep='::', header=None, encoding='ISO-8859-1')


In [83]:
attrs_df['sid'] = attrs_df['sid'].map(u_i_map['sid'])

In [87]:
attrs_df

,sid,genre
0,0.0,Animation|Children's|Comedy
1,1.0,Adventure|Children's|Fantasy
2,2.0,Comedy|Romance
3,3.0,Comedy|Drama
4,4.0,Comedy
...,...,...
3878,3701.0,Comedy
3879,3702.0,Drama
3880,3703.0,Drama
3881,3704.0,Drama


In [88]:
attrs_df, amap = dataset.densify_index(attrs_df, target_cols=dataset.attr_names)

Densifying index


In [91]:
i2attr_map = {}
for attr_name in dataset.attr_names:
    i2attr_map[attr_name] = dict(zip(attrs_df['sid'], attrs_df[attr_name]))